# RAG与Memory - 简单例子

## 什么是RAG？

RAG（Retrieval-Augmented Generation）是一种结合了检索和生成的AI技术。

**基本原理：**
1. **存储阶段**：将知识/记忆转换为向量嵌入，存储在向量数据库中
2. **检索阶段**：用户查询时，将查询转换为向量，在数据库中搜索相似的内容
3. **生成阶段**：将检索到的相关信息作为上下文，生成回答

**优势：**
- 可以动态添加新知识，无需重新训练模型
- 提供可追溯的信息来源
- 减少模型幻觉

在这个例子中，我们将使用Pinecone作为向量数据库，存储用户的个人偏好和信息。
## 注意：Pinecone 目前可能已经不是向量数据库的最优选择，请务必谨慎使用！

In [20]:

import os
from dotenv import load_dotenv
from pinecone import Pinecone
from datetime import datetime
from pinecone import IndexEmbed

load_dotenv()

MEMORY_SCORE_THRESHOLD = 0.1

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

index_name = "bytedance-agents101"
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed=IndexEmbed(
            model="multilingual-e5-large",
            field_map={"text": "chunk_text"},
            metric="cosine"
        )
    )
index = pc.Index(index_name)

def embed_text(text, input_type="passage"):
    """将文本转换为向量嵌入"""
    return pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[text],
        parameters={"input_type": input_type, "truncate": "END"}
    )

def add_memory(memory_text, memory_id=None):
    """添加记忆到向量数据库"""
    print(f"添加记忆: {memory_text}")
    
    # 生成嵌入
    embeddings = embed_text(memory_text)
    
    # 生成ID（如果没有提供）
    if not memory_id:
        memory_id = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    
    # 存储到Pinecone
    index.upsert(
        vectors=[{
            "id": memory_id, 
            "values": embeddings[0]['values'], 
            "metadata": {"text": memory_text}
        }]
    )
    print(f"记忆已存储，ID: {memory_id}")
    return memory_id

def query_memory(query, top_k=10):
    """查询相关记忆"""
    print(f"查询: {query}")
    
    query_embedding = embed_text(query, input_type="query")
    
    query_results = index.query(
        vector=query_embedding[0].values,
        top_k=top_k,
        include_metadata=True,
        include_values=False
    )
    
    results = []
    for result in query_results['matches']:
        if result['score'] >= MEMORY_SCORE_THRESHOLD:
            results.append({
                "id": result['id'],
                "content": result['metadata']['text'],
                "score": result['score']
            })
    
    return results


In [40]:
# 添加样例记忆
sample_memories = [
    "用户是中国人",
    "用户在中国喜欢肯德基和麦当劳，但在美国不喜欢肯德基和麦当劳",
    "用户比起鸡肉更喜欢牛肉",
    "用户在TikTok上班",
    "用户喜欢先给一句简短的答案，再给详细解释",
    "用户喜欢听Michael Jackson的歌",
]

memory_ids = []
for i, memory in enumerate(sample_memories):
    memory_id = add_memory(memory, f"sample_{i+1}")
    memory_ids.append(memory_id)
    print()

print(f"共添加了 {len(memory_ids)} 条记忆")


添加记忆: 用户是中国人
记忆已存储，ID: sample_1

添加记忆: 用户在中国喜欢肯德基和麦当劳，但在美国不喜欢肯德基和麦当劳
记忆已存储，ID: sample_2

添加记忆: 用户比起鸡肉更喜欢牛肉
记忆已存储，ID: sample_3

添加记忆: 用户在TikTok上班
记忆已存储，ID: sample_4

添加记忆: 用户喜欢先给一句简短的答案，再给详细解释
记忆已存储，ID: sample_5

添加记忆: 用户喜欢听Michael Jackson的歌
记忆已存储，ID: sample_6

共添加了 6 条记忆


In [41]:
queries = [
    "炸鸡",
    "为什么天空这样蓝",
    "炸鸡健康还是烤牛肉串健康",
    "ByteDance",
    "Musically",
    "musical",
    "MacBook Pro",
    "San Jose is a hot place in summer",
    "疯狂星期四",
    "Cause this is thriller, thriller night, and no one's gonna save you from the beast that's about to strike"
]

print("=== 查询例子 ===")
for query in queries:
    print(f"\n🔍 查询：{query}")
    print("-" * 50)
    
    results = query_memory(query)
    
    if results:
        print(f"找到 {len(results)} 条相关记忆:")
        for i, result in enumerate(results, 1):
            print(f"{i}. [{result['score']:.3f}] {result['content']}")
    else:
        print("没有找到相关记忆（相似度低于阈值）")
    
    print()


=== 查询例子 ===

🔍 查询：炸鸡
--------------------------------------------------
查询: 炸鸡
找到 6 条相关记忆:
1. [0.816] 用户比起鸡肉更喜欢牛肉
2. [0.811] 用户在中国喜欢肯德基和麦当劳，但在美国不喜欢肯德基和麦当劳
3. [0.809] 用户喜欢先给一句简短的答案，再给详细解释
4. [0.795] 用户在TikTok上班
5. [0.783] 用户是中国人
6. [0.782] 用户喜欢听Michael Jackson的歌


🔍 查询：为什么天空这样蓝
--------------------------------------------------
查询: 为什么天空这样蓝
找到 6 条相关记忆:
1. [0.845] 用户喜欢先给一句简短的答案，再给详细解释
2. [0.825] 用户在中国喜欢肯德基和麦当劳，但在美国不喜欢肯德基和麦当劳
3. [0.822] 用户在TikTok上班
4. [0.818] 用户比起鸡肉更喜欢牛肉
5. [0.814] 用户是中国人
6. [0.809] 用户喜欢听Michael Jackson的歌


🔍 查询：炸鸡健康还是烤牛肉串健康
--------------------------------------------------
查询: 炸鸡健康还是烤牛肉串健康
找到 6 条相关记忆:
1. [0.860] 用户比起鸡肉更喜欢牛肉
2. [0.825] 用户喜欢先给一句简短的答案，再给详细解释
3. [0.819] 用户在中国喜欢肯德基和麦当劳，但在美国不喜欢肯德基和麦当劳
4. [0.801] 用户在TikTok上班
5. [0.795] 用户是中国人
6. [0.788] 用户喜欢听Michael Jackson的歌


🔍 查询：ByteDance
--------------------------------------------------
查询: ByteDance
找到 6 条相关记忆:
1. [0.737] 用户喜欢先给一句简短的答案，再给详细解释
2. [0.736] 用户在TikTok上班
3. [0.718] 用户喜欢听Michael Jackson的歌
4. [0.715] 用户是中国人
5.

## RAG的语义捕捉能力与局限性

从上面的查询结果可以看出，RAG系统具有一定的语义理解能力，但也存在明显的局限性：

### ✅ RAG能够很好捕捉的语义：

1. **直接关联** - "炸鸡" 能找到肯德基相关记忆，"牛肉"偏好
2. **隐含关联** - "ByteDance" -> TikTok，"疯狂星期四" -> 肯德基，Musically -> TikTok，"为什么天空这样蓝" -> 对解释风格的偏好

### ❌ RAG难以准确捕捉的语义：

1. **多步推理（世界知识）** - "Cause this is thriller, thriller night..." 这段Michael Jackson经典歌词，虽然能检索到"用户喜欢听Michael Jackson的歌"，但相关性分数只有0.725，排在第4位，说明RAG无法很好地识别具体的歌词内容与歌手偏好的关联

2. **上下文理解** - 无法像人类一样理解复杂的语境和隐含意义

### 🎯 实际应用建议：

- **适用场景**：知识问答、个人偏好匹配、相关内容推荐
- **局限场景**：需要深度推理、复杂语境理解、创意内容生成 （改进：使用“Agentic RAG”）

总结：RAG是一个强大的语义检索工具，但不是万能的。


## 例子：能记住用户偏好的聊天机器人（现ChatGPT的实现）

In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

USE_RAG = True

def chatbot(user_message):
    """简单的聊天机器人函数"""
    if USE_RAG:
        relevant_memories = query_memory(user_message, top_k=3)
        if relevant_memories:
            memory_context = "\n".join([f"- {mem['content']}" for mem in relevant_memories])
            system_prompt = f"""你是一个智能助手。根据以下用户偏好信息来回答问题：

用户偏好信息：
{memory_context}

请基于这些信息给出个性化的回答。"""
        else:
            system_prompt = "你是一个智能助手。"
            
        messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_message)]
    else:
        # 不使用RAG，直接回答
        messages = [HumanMessage(content=user_message)]
    
    response = llm.invoke(messages)
    return response.content


In [42]:
user_question = "人在美国，刚下飞机，推荐几个快餐吃"

In [43]:
USE_RAG = True
response = chatbot(user_question)
print(response)

查询: 人在美国，刚下飞机，推荐几个快餐吃
推荐 Five Guys、Shake Shack 或 In-N-Out（如果在西海岸）。

**详细解释：**

考虑到您在美国不喜欢肯德基和麦当劳，我为您推荐其他受欢迎且以牛肉产品为主的快餐选择：

*   **Five Guys：** 这是一家遍布全美的汉堡连锁店，以新鲜牛肉汉堡和可以自由选择的丰富配料闻名。他们的薯条也很受欢迎，通常会给很多。
*   **Shake Shack：** 起源于纽约，现在在美国许多地方都有分店。他们提供高品质的安格斯牛肉汉堡、热狗、奶昔等，口感和食材都比一般快餐更胜一筹。
*   **In-N-Out Burger：** 如果您在美国西海岸（加州、亚利桑那、内华达、犹他、德州、俄勒冈、科罗拉多），这是非常受欢迎的本土选择。他们以简洁的菜单、新鲜的牛肉汉堡和独特的“秘密菜单”而闻名，是体验美式快餐文化的不错选择。


In [44]:
USE_RAG = False
response = chatbot(user_question)
print(response)

欢迎来到美国！刚下飞机肯定又累又饿，快餐是最好的选择。这里有几家非常适合刚下飞机、想快速解决温饱的快餐店推荐：

1.  **McDonald's (麦当劳)**
    *   **特点：** 全球连锁，普及率最高，几乎每个机场附近、每个城市都有。菜单熟悉，点餐方便，速度快。
    *   **推荐理由：** 如果你只想最快、最方便地吃到东西，麦当劳绝对是首选。虽然不是最有特色的，但胜在稳定和便捷。
    *   **必试：** Big Mac (巨无霸), Quarter Pounder with Cheese (四盎司牛肉堡), Chicken McNuggets (麦乐鸡)。

2.  **Chick-fil-A (福来鸡)**
    *   **特点：** 美国本土非常受欢迎的炸鸡快餐连锁，以其优质的鸡肉三明治、薯条和卓越的客户服务闻名。食物品质普遍高于其他快餐店。
    *   **推荐理由：** 如果你想尝试一些在美国非常流行且口碑很好的快餐，Chick-fil-A是绝佳选择。他们的鸡肉三明治被很多人认为是快餐界的“天花板”。
    *   **必试：** Chicken Sandwich (原味鸡肉三明治), Waffle Fries (格子薯条), Chick-fil-A Sauce (福来鸡酱)。
    *   **注意：** 他们**周日不营业**！所以如果是周日抵达，就去不了啦。

3.  **Chipotle Mexican Grill (奇波特墨西哥烧烤)**
    *   **特点：** 主打新鲜、健康、定制化的墨西哥风味快餐。你可以选择卷饼 (Burrito) 或碗 (Burrito Bowl)，然后自己选择米饭、豆类、肉类（鸡肉、牛肉、猪肉、素食）、蔬菜和各种酱料。
    *   **推荐理由：** 如果你不想吃传统的汉堡薯条，想吃点“健康”一点、口味更丰富的，Chipotle 是个好选择。点餐流程简单，食物新鲜。
    *   **必试：** Burrito Bowl (碗，没有饼皮，更健康), Guacamole (鳄梨酱，虽然要额外收费，但很值得)。

4.  **Wendy's (温蒂汉堡)**
    *   **特点：** 美国第三大汉堡连锁，以其方形的牛肉饼和新鲜的食材（号称“从不冷冻”）闻名。
  

## 在我们的项目中的可能用法

1. Procedural memory
    - 先这样，然后再这样，最后再这样
2. 积累特定知识
    - 劲爆的视频往往有一个平静的开始，然后突然爆发
    - 情绪向的视频往往...
3. 过往成功案例
    - 直接模仿以前的example，动态加载示例
4. 从成千上万的素材里面retrieve几个最贴合的素材
    - 例如，每种特效有一段自然语言描述，难以用规则搜索

## Agentic RAG
当简单的RAG无法满足需求时，可以考虑使用Agentic RAG，即将RAG作为LLM的工具，让LLM来决定如何使用RAG。

**关键区别：**
- **简单RAG**：每次查询都自动检索记忆
- **Agentic RAG**：让LLM判断是否需要检索，以及如何检索

**适用场景：**
- 用户输入很长，包含多个话题
- 需要多次检索不同关键词  
- 需要根据上下文决定是否检索

**优势：**
- 更智能的检索决策
- 可以处理复杂的多步推理
- 减少不必要的检索调用

# 仅作示例，跑不动

In [ ]:
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

@tool(parse_docstring=True)
def search_memory(query: str, top_k: int = 3) -> str:
    """搜索用户的历史记忆和偏好信息
    
    Args:
        query: 搜索查询词
        top_k: 返回结果数量
    
    Returns:
        相关记忆信息的字符串
    """
    results = query_memory(query, top_k=top_k)
    if not results:
        return "没有找到相关记忆信息"
    
    memory_text = "\n".join([f"- {mem['content']}" for mem in results])
    return f"找到相关记忆信息:\n{memory_text}"

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
llm_with_tools = llm.bind_tools([search_memory])

system_prompt = """你是一个智能助手。当用户提问时，如果你认为需要了解用户的历史偏好或相关信息，请先使用search_memory工具搜索相关记忆。

请基于搜索到的信息给出个性化的回答。"""

# 模拟用户长输入
user_input = """
刚下飞机...[省略100字]...San Jose的机场真的好小啊...[省略100字]...真的快饿死了...
"""

# 调用Agentic RAG
def agentic_rag_chatbot(user_message):
    from langchain_core.messages import SystemMessage, HumanMessage
    
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message)
    ]
    
    response = llm_with_tools.invoke(messages)
    # 用户现在需要吃东西，我可以搜索关于用户对饮食的偏好
    # call search_memory("吃饭 饮食 快餐")
    if response.tool_calls:
        ...
    else:
        return response.content
